<h2 style="color:orange;font-weight:bold;font-family:arial">Email Spam Detection and Filtering System</h2>
<h3 style="color:blue;font-weight:bold;font-family:arial">Samruddhi Khairnar</h3>

##### API Keys in credentials.json, of my gmail account - kbtug20170@kbtcoe.org

In [ ]:
##### pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

#### **Imports**

In [29]:
import os.path
import pickle
import spacy
sp = spacy.load('en_core_web_md')

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from base64 import urlsafe_b64decode, urlsafe_b64encode

#### **Scope to grant all permissions to access GMail Account using API**

In [30]:
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

#### **Variables**

In [38]:
creds = None
flow = None
text = []
ids = []

#### **Using Credentials to Login**

In [39]:
if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

#### **Getting All Emails from Inbox**

In [40]:
try:
    service = build('gmail', 'v1', credentials=creds)
    results = service.users().messages().list(userId='me').execute()
    messages = results.get('messages', [])

except HttpError as error:
    print(f'An error occurred: {error}')

#### **Loading first 3 Emails to check for spam**
##### (Only 3 to avoid any risk, regarding the emails I've received)

In [41]:
try:
    for msg in messages[1:3]:  
        msg = service.users().messages().get(userId='me', id=msg['id']).execute()
        parts = msg['payload'].get("parts")
        txt = ''
        for part in parts:
            mimeType = part.get("mimeType")
            data = part.get("body").get("data")
            if mimeType == "text/plain": #or html
                txt += urlsafe_b64decode(data).decode()
        text.append(txt)
        ids.append(msg['id'])
        print(msg['id'])

except HttpError as error:
    print(f'An error occurred: {error}')

187c068d19f1f75d
187c058626a5b5ff


#### **Loading the Saved Model and Checking for Spam**

In [44]:
with open('email_spam_filter.pkl','rb') as f:
    loaded = pickle.load(f)
    pred = loaded.predict([sp(t).vector for t in text])
    print(pred)

['non-spam' 'spam']


#### **Listing Emails marked as Spam**

In [45]:
spam_list = []
for i, t in enumerate(pred):
    if(t=='spam'):
        spam_list.append(ids[i])
print(spam_list)

['187c058626a5b5ff']


#### **Trashing the Listed Emails**

In [46]:
try:
    for s in spam_list:
        service.users().messages().trash(userId='me', id=s).execute()
        print('Deleted spam email with id = ',s)

except HttpError as error:
    print(f'An error occurred: {error}')

Deleted spam email with id =  187c058626a5b5ff
